In [1]:
# Standard imports
import os

# Third-party imports
import cv2

# Local imports
from bbox_painter import BBoxPainter
from parking_detector import ParkingDetector

## Extract the first frame for bbox annotating purpose

In [ ]:
vid_filepath = '../data/parking1.mp4'
extract_filepath = None

def extract_img():
    vid = cv2.VideoCapture(vid_filepath)  
    
    ret, frame = vid.read()
    if ret:
        img = os.path.basename(vid_filepath).rsplit('.', 1)[0] + '.jpg'
        extract_filepath = os.path.join('../output', img)
        
        os.makedirs(os.path.dirname(extract_filepath), exist_ok=True)
        cv2.imwrite(extract_filepath, frame)
    
    vid.release()

In [ ]:
extract_img()

## Manually define the bboxes for parking slots

In [4]:
painter = BBoxPainter()
json_filepath = painter.get_json_filepath() or '../output/bboxes.json'

In [5]:
# classes - https://gist.github.com/rcland12/dc48e1963268ff98c8b2c4543e7a9be8
parking = ParkingDetector(model='yolo11s.pt', source=vid_filepath, json_file=json_filepath) 

requirements: Ultralytics requirement ['shapely>=2.0.0'] not found, attempting AutoUpdate...
   ---------------------------------------- 1.7/1.7 MB 23.2 MB/s eta 0:00:00

requirements: AutoUpdate success  6.0s
WARNING requirements: Restart runtime or rerun command for updates to take effect

Ultralytics Solutions:  {'source': '../data/parking1.mp4', 'model': 'yolo11s.pt', 'classes': None, 'show_conf': True, 'show_labels': True, 'region': None, 'colormap': 21, 'show_in': True, 'show_out': True, 'up_angle': 145.0, 'down_angle': 90, 'kpts': [6, 8, 10], 'analytics_type': 'line', 'figsize': (12.8, 7.2), 'blur_ratio': 0.5, 'vision_point': (20, 20), 'crop_dir': 'cropped-detections', 'json_file': None, 'line_width': 2, 'records': 5, 'fps': 30.0, 'max_hist': 5, 'meter_per_pixel': 0.05, 'max_speed': 120, 'show': False, 'iou': 0.7, 'conf': 0.25, 'device': None, 'max_det': 300, 'half': False, 'tracker': 'botsort.yaml', 'verbose': True, 'data': 'images'}


100%|█████████████████████████████████████████████████████████████████████████████| 18.4M/18.4M [00:00<00:00, 24.3MB/s]


## Output processed video to data/output.mp4

In [6]:
def output():
    vid = cv2.VideoCapture(vid_filepath)
    fps = vid.get(cv2.CAP_PROP_FPS)
    frame_interval = int(1000/fps) # ms
    
    ret, frame = vid.read()
    output_vid = cv2.VideoWriter('../output/output.mp4', cv2.VideoWriter_fourcc(*'MP4V'), frame_interval, (frame.shape[1], frame.shape[0]))
    
    while ret:
        frame = parking.process_frame(frame)
        output_vid.write(frame)
        ret, frame = vid.read()
    
    vid.release()
    output_vid.release()

In [7]:
output()